# Define Event

Before we can analyze the event, we need to identify the boundaries of the event. We will accomplish this by

1. Defining the values to be classified as gains and losses
2. Creating a counts table defining the various events
3. Defining the porportion of patients with gain or loss to be considered significant
4. Identifying regions of gain and loss
5. Identifying regions where all cancer types meet criteria for gain or loss event.

## Setup

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import cptac
import numpy as np
import cnvutils

cptac warning: Your version of cptac (0.8.5) is out-of-date. Latest is 0.8.6. Please run 'pip install --upgrade cptac' to update it. (<ipython-input-1-7c050b7b6e1e>, line 4)


## Part 1: Define Parameters

These are the parameters that must be set for each analysis.

In [2]:
# # A list of cancer types to look at (should be all capital letter ex: BRCA)
# CANCER_TYPES = ['BRCA', 'COLON', 'HNSCC', 'LSCC', 'LUAD', 'OVARIAN']
# The porportion of patients with gain or loss of a given gene to be considered
PATIENT_CUTOFF = 0.2
# The chromosome to be analyzed (should be a string)
CHROMOSOME = '13'
# The arm to be analyzed. Options are: 'p', 'q' or 'both'.
ARM = 'q'

In [3]:
# Here you will need to load in all the cancer types to be considered. We have listed all for convenience, simply comment
# out the cancer types you do not want to consider.
cnv = {
    'BRCA': cptac.Brca().get_CNV(),
#     'CCRCC': cptac.Ccrcc().get_CNV(),
#     'COLON': cptac.Colon().get_CNV(),
#     'ENDO': cptac.Endometrial().get_CNV(),
    'GBM': cptac.Gbm().get_CNV(),
    'HNSCC': cptac.Hnscc().get_CNV(),
    'LSCC': cptac.Lscc().get_CNV(),
    'LUAD': cptac.Luad().get_CNV(),
    'OVARIAN': cptac.Ovarian().get_CNV()
}

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (<ipython-input-3-a1fe5222d684>, line 8)


cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (<ipython-input-3-a1fe5222d684>, line 9)


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (<ipython-input-3-a1fe5222d684>, line 10)


## Part 2: Determine Cutoffs

In [4]:
counts = cnvutils.get_counts_table()

C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
cutoffs = dict()
for cancer_type in cnv.keys():
    df = cnv[cancer_type]
    cutoffs[cancer_type] = len(df) * PATIENT_CUTOFF

## Part 3: Subset Event

In [6]:
# counts.cancer.unique()

In [7]:
counts = counts[(counts.chromosome == CHROMOSOME)]
if ARM != 'both':
    counts = counts[counts.arm == ARM]

In [8]:
# counts

## Find Loss Regions

In [9]:
df = counts
loss_event_locations = dict()
for cancer in cnv.keys():
    df_loss = df[(df.variable == 'loss') & (df.cancer == cancer)].sort_values('start_bp')
    print(len(df_loss))
    values = list(df_loss.value)
    loss_events = list()
    start = None
    for i in range(0, len(values)):
        val = values[i]
        if val > cutoffs[cancer]:
            if start is None:
                start = i
        else:
            if start is not None:
                loss_events.append((start, i))
                start = None
    if start is not None:
        loss_events.append((start, len(values)-1))
    event_locations = list()
    for event in loss_events:
        start_bp = df_loss.iloc[event[0]].start_bp
        end_bp = df_loss.iloc[event[1]].start_bp
        event_locations.append((start_bp, end_bp-start_bp))
    loss_event_locations[cancer] = event_locations

515
310
1335
543
298
268


In [10]:
loss_event_locations['LUAD']

[(19173772.0, 95107829.0)]

In [11]:
loss_event_patients = list()
for cancer in loss_event_locations.keys():
    events = loss_event_locations[cancer]
    for event in events:
        start = event[0]
        end = event[0] + event[1]
        loss_event_patients.append((start, 1))
        loss_event_patients.append((end, 0))
#     patients += list(gain_event_locations[cancer])
loss_event_patients.sort()

In [12]:
counts

,Name,Database_ID,start_bp,end_bp,cancer,chromosome,arm,variable,value
75,ABCC4,ENSG00000125257.9,95019835.0,95301475.0,BRCA,13,q,gain,12
100,ABHD13,ENSG00000139826.5,108218392.0,108234243.0,BRCA,13,q,gain,18
101,ABHD13,ENSG00000139826.5,108218392.0,108234243.0,CCRCC,13,q,gain,5
399,ADPRHL1,ENSG00000153531.8,113399610.0,113453524.0,BRCA,13,q,gain,21
532,AKAP11,ENSG00000023516.7,42272152.0,42323261.0,BRCA,13,q,gain,8
...,...,...,...,...,...,...,...,...,...
476486,ZDHHC20P4,NaN,68985058.0,68986064.0,HNSCC,13,q,loss,23
476487,ZDHHC4P1,NaN,42933858.0,42934892.0,HNSCC,13,q,loss,28
476531,ZNF646P1,NaN,53408882.0,53409876.0,HNSCC,13,q,loss,26
476570,ZNF962P,NaN,18467172.0,18485448.0,HNSCC,13,q,loss,27


In [13]:
count = 0
current_bp = 0
start = list()
end = list()
size = list()
total = list()
for patient in loss_event_patients:
    if patient[0] != current_bp:
        start.append(current_bp)
        end.append(patient[0])
        size.append(patient[0]-current_bp)
        total.append(count)
        current_bp = patient[0]
    if patient[1] == 1:
        count += 1
    else:
        count -= 1
loss_event_data = pd.DataFrame({'start': start, 'end': end, 'counts': total, 'length': size})

In [14]:
loss_event_data

,start,end,counts,length
0,0.0,18174010.0,0,18174010.0
1,18174010.0,18195297.0,1,21287.0
2,18195297.0,19173772.0,3,978475.0
3,19173772.0,19422877.0,5,249105.0
4,19422877.0,27255401.0,6,7832524.0
5,27255401.0,27270327.0,5,14926.0
6,27270327.0,45337480.0,6,18067153.0
7,45337480.0,45393316.0,5,55836.0
8,45393316.0,71296210.0,6,25902894.0
9,71296210.0,110148963.0,5,38852753.0


## Find Gain Regions

In [15]:
# df = counts
# gain_event_locations = dict()
# for cancer in cnv.keys():
#     df_gain = df[(df.variable == 'gain') & (df.cancer == cancer)].sort_values('start_bp')
#     values = list(df_gain.value)
#     gain_events = list()
#     start = None
#     for i in range(0, len(values)):
#         val = values[i]
#         if val > cutoffs[cancer]:
#             if start is None:
#                 start = i
#         else:
#             if start is not None:
#                 gain_events.append((start, i))
#                 start = None
#     if start is not None:
#         gain_events.append((start, len(values)-1))
#     event_locations = list()
#     for event in gain_events:
#         start_bp = df_gain.iloc[event[0]].start_bp
#         end_bp = df_gain.iloc[event[1]].start_bp
        
#         event_locations.append((start_bp, end_bp-start_bp))
#     gain_event_locations[cancer] = event_locations

In [16]:
# gain_event_patients = list()
# for cancer in gain_event_locations.keys():
#     events = gain_event_locations[cancer]
#     for event in events:
#         start = event[0]
#         end = event[0] + event[1]
#         gain_event_patients.append((start, 1))
#         gain_event_patients.append((end, 0))
# #     patients += list(gain_event_locations[cancer])
# gain_event_patients.sort()

In [17]:
# count = 0
# current_bp = 0
# # results = list()
# start = list()
# end = list()
# size = list()
# total = list()
# for patient in gain_event_patients:
#     if patient[0] != current_bp:
#         start.append(current_bp)
#         end.append(patient[0])
#         size.append(patient[0]-current_bp)
#         total.append(count)
# #         results.append((current_bp, patient[0], count))
#         current_bp = patient[0]
#     if patient[1] == 1:
#         count += 1
#     else:
#         count -= 1
# gain_event_data = pd.DataFrame({'start': start, 'end': end, 'counts': total, 'length': size})

In [18]:
# gain_event_data